## Import modules

In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np

## Read all files

In [ ]:
"""
btcFiles =  [
            "normal-old",
            #"normal-new",
            "heikin-old",
            #"heikin-new",
            "onchain-old",
            #"onchain-new"
            ]
"""
restFiles = [
            "NASDAQ_DLY NDX, 1D",
            "OANDA XAUUSD, 1D", 
            "SP SPX, 1D", 
            "TVC DXY, 1D"
            ]
ethFiles =  [
            "BITSTAMP ETHUSD, 1D",
            #"BITSTAMP ETHUSD, 1D-otheronchain-data"
            ]

### create dataframe for each file 

pack all dataframes belonging to one topic in one list

In [ ]:
#btcData = [pd.read_csv("daten/btc/"+filename+".csv") for filename in btcFiles]
btcData = [pd.read_csv("btcAllData-inclusiveDate-inclusiveHalving.csv")]
ethData = [pd.read_csv("daten/eth/"+filename+".csv") for filename in ethFiles]
restData = [pd.read_csv("daten/rest/"+filename+".csv") for filename in restFiles]

### get overview over Dataframes of each file

In [ ]:
[print(data.info()) for data in btcData]

In [ ]:
[print(data.info()) for data in ethData]

In [ ]:
[print(data.info()) for data in restData]

### check time ranges of all data

- btc data
    - starts at 2011-08-18
    - ends at 2022-06-12
- rest data
    - starts earlier
    - ends on 01.06.22
- eth data
    - starts at 2015-08-07
    - end at 2022-06-12
    
$\Rightarrow $ 

start: earliest entry of bitcoin

end: 01.06.2022

be careful with etherum data because of later start

In [ ]:
data = [*btcData, *restData, *ethData]
for df in data:
    print(datetime.fromtimestamp(df['time'].iloc[0]), datetime.fromtimestamp(df['time'].iloc[-1]))

#### look for missing entries

- btc Data 
    - only 3 days left 
    - just interpolate via linear regression
 
- eth data
    - all perfect
    
- rest data 
    - seems like it is downloaded on wrong time series (too long intervals)

In [ ]:
def checkMissingData(df, stepwidth=24*60*60, leaveFirstn=200):
    startDate = df['time'].iloc[leaveFirstn]
    endDate = df['time'].iloc[-1]
    numberSteps = len(df.index)- leaveFirstn -1
    expectedNumberSteps = (endDate-startDate)/ (stepwidth)
    print("neues dataframe", "\n",
          "start ",datetime.fromtimestamp(startDate), startDate, "\n",
          "end   ", datetime.fromtimestamp(endDate), endDate,"\n",
          "is ", numberSteps,"should ", expectedNumberSteps, "\n",
          "number missing" , expectedNumberSteps - numberSteps,
          "\n -------------------------------------------------------------")
    startMissingIndice= []; rangeMissingIndice= []
    for j in range(leaveFirstn, numberSteps-1):
        timestamp = df['time'][j+1]
        ascendantTimestamp = df['time'][j]
        if (timestamp - ascendantTimestamp) != stepwidth:
            numberMissing = (timestamp - ascendantTimestamp) / stepwidth - 1
            print(j, numberMissing,
                  datetime.fromtimestamp(ascendantTimestamp), datetime.fromtimestamp(timestamp))
            startMissingIndice.append(j); rangeMissingIndice.append(numberMissing)
    print("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")
    return startMissingIndice, rangeMissingIndice

In [ ]:
for df in btcData:
    checkMissingData(df)

## Observation 

- 3 days missing in data at 2015-01-09 
[Reason is a Hack](https://venturebeat.com/2015/01/09/bitstamp-resumes-trading-after-5m-in-losses-led-to-shutdown/)

- more values missing in old data but only in the first 200 days of bitcoin (we do not look at those)-  

- old data is on daily timeframe and goes from (2011, 8, 18, 2, 0) to (2022, 6, 12, 2, 0) but the first 200 days are left out because of missing ema, sma, ...


#### linear regression over missing values

In [ ]:
def calculateMissingValues(start,end,missingValues=3):
    stepwidth = (end- start) / (missingValues + 1)
    calculatedValues= [start + i* stepwidth for i in range(1,missingValues+1)]
    return calculatedValues

In [ ]:
def fillInValues(startValue,endValue, fillInValue):
    return [fillInValue for i in range(endValue-startValue+1)]

## pimp Bitcoin Data and create one file with all information

In [ ]:
# 24H volume is not tracked at the beginning so it is set to zero
for i in range(0,2):
    df = btcData[i]
    df['24H Volume'] = df['24H Volume'].fillna(0)
    # delete those two columns beacuse they are always nan
    #df.drop('Upper Bollinger Band', inplace=True, axis=1)
    #df.drop('Lower Bollinger Band', inplace=True, axis=1)
    # delete first 200 rows because ema200, sma200 is nan there
    # and the first few days are completely unreliable
    df.drop(df.index[0:200],inplace=True, axis=0)

finalBtcData = btcData[0]
for i in range(1,len(btcData)):
    finalBtcData = pd.merge(finalBtcData, btcData[i], how="left", on="time")

### Jetzt sind die Daten konsistent aber es fehlten noch die 3 Einträge 

In [ ]:
checkMissingData(finalBtcData)

#checkMissingData(df)   #this method has to be implemented new for dataframes instead of files 
#for start in startIndex:
#    startValue = None
    #calculateMissingValues(startValue,endValue,missingValues)

#### insert missingData in btcData

In [ ]:
insertion_point = 1007
numberColumns = len(finalBtcData.columns)
# go through columns
fillInDataArray = [[0 for j in range(numberColumns)] for i in range(3)]
for i in range(numberColumns):
    valuesForColumn = calculateMissingValues(finalBtcData.iloc[:, i][1006], finalBtcData.iloc[:,i][1007],3)
    for j in range(3):
        fillInDataArray[j][i] = valuesForColumn[j]
fillInDataArray = pd.DataFrame(np.array(fillInDataArray), columns= finalBtcData.columns)
finalBtcData = pd.concat([finalBtcData.iloc[:insertion_point], fillInDataArray, finalBtcData.iloc[insertion_point:]]).reset_index(drop=True)

fillInDataArray

In [ ]:
finalBtcData.info()

In [ ]:
checkMissingData(finalBtcData, stepwidth=24*60*60, leaveFirstn=0)

In [ ]:
#finalBtcData["date"] = datetime.fromtimestamp(finalBtcData["time"])
finalBtcData["date"] = [datetime.fromtimestamp(x) for x in finalBtcData["time"]]
finalBtcData["year"] = [datetime.fromtimestamp(x).year for x in finalBtcData["time"]]
finalBtcData["month"] = [datetime.fromtimestamp(x).month for x in finalBtcData["time"]]
finalBtcData["dayInMonth"] = [datetime.fromtimestamp(x).day for x in finalBtcData["time"]]
finalBtcData["weekday"] = [datetime.fromtimestamp(x).weekday() for x in finalBtcData["time"]]
finalBtcData.head()

In [ ]:
finalBtcData.to_csv('btcAllData-inclusiveDate.csv',index=False)

## Merge Btc and Eth

In [ ]:
btcData = pd.read_csv("btcAllData-inclusiveDate-inclusiveHalving.csv")
ethData = pd.read_csv("daten/eth/BITSTAMP ETHUSD, 1D.csv")
finalBtcData = pd.merge(btcData, ethData, how="left", on="time")
finalBtcData = finalBtcData.fillna(0)
finalBtcData.head()
finalBtcData.to_csv('btcAllData-inclusiveEth.csv',index=False)

### merge btceth and restfiles

In [31]:
import pandas as pd
from datetime import datetime
import numpy as np

restFiles = [
            "NASDAQ_DLY NDX, 1D",
            "OANDA XAUUSD, 1D", 
            "SP SPX, 1D", 
            "TVC DXY, 1D"
            ]
restData = [pd.read_csv("daten/rest/"+filename+".csv") for filename in restFiles]

btcethdata = pd.read_csv("daten/btceth.csv")
                 

# merge data
# for each file
for i in range(len(restData)):
    restData[i]["dateShort"] = [datetime.fromtimestamp(x).date() for x in restData[i]["time"]]
    del restData[i]["time"]
    # for each columns
    for j in range(len(restData[i].columns)):
        # rename columns with filename included
        if restData[i].columns.values[j] != "dateShort":
            restData[i].columns.values[j] = (restFiles[i]+ " " + (restData[i].columns[j]))

#merge all files together
btcethdata["dateShort"] = [datetime.fromtimestamp(x).date() for x in btcethdata["time"]]
finalBtcData = btcethdata
for i in range(len(restData)):
    finalBtcData = pd.merge(finalBtcData, restData[i], how="left", on="dateShort")
finalBtcData.head()
finalBtcData = finalBtcData.fillna(method='ffill')
finalBtcData.to_csv("test2.csv", index=False)
finalBtcData.head()

,time,open_x,high_x,low_x,close_x,SMA20_x,SMA50_x,SMA100_x,SMA2000_x,Regression Line_x,...,"TVC DXY, 1D low","TVC DXY, 1D close","TVC DXY, 1D EMA20","TVC DXY, 1D EMA50","TVC DXY, 1D EMA100","TVC DXY, 1D EMA200","TVC DXY, 1D SMA20","TVC DXY, 1D SMA50","TVC DXY, 1D SMA100","TVC DXY, 1D SMA2000"
0,1333497600,4.92,5.01,4.86,5.01,4.9310,4.8948,5.4069,4.98145,4.722802,...,79.386,79.740,79.371898,79.379826,79.063660,78.410082,79.54070,79.24766,79.51609,77.720130
1,1333584000,4.99,5.04,4.89,5.00,4.9180,4.9010,5.4140,4.94800,4.684430,...,79.623,80.050,79.436479,79.406107,79.083191,78.426400,79.58510,79.26072,79.53379,77.748485
2,1333670400,4.90,4.99,4.81,4.99,4.8965,4.9106,5.4209,4.91445,4.645442,...,79.736,79.886,79.479291,79.424926,79.099089,78.440923,79.57735,79.28040,79.55201,77.776425
3,1333756800,4.94,4.94,4.70,4.70,4.8690,4.9112,5.4229,4.87945,4.598616,...,79.736,79.886,79.479291,79.424926,79.099089,78.440923,79.57735,79.28040,79.55201,77.776425
4,1333843200,4.80,4.80,4.69,4.76,4.8655,4.9218,5.4284,4.84475,4.548236,...,79.652,79.795,79.509358,79.439439,79.112869,78.454397,79.57475,79.29362,79.56647,77.804180


In [25]:
import pandas as pd
from datetime import datetime
import numpy as np

df = pd.read_csv("daten/rest/NASDAQ_DLY NDX, 1D.csv")
df = df.fillna(method='ffill')
df.head()

,time,open,high,low,close,EMA20,EMA50,EMA100,EMA200,SMA20,SMA50,SMA100,SMA2000
0,1294410600,2280.12988,2282.87012,2254.21997,2276.69995,2232.869701,2179.905564,2103.514096,2004.474299,2233.036499,2187.042207,2071.960108,1983.078022
1,1294669800,2270.64990,2288.63989,2265.00000,2284.37988,2237.775433,2184.002596,2107.095597,2007.259429,2236.488489,2190.135205,2076.295906,1984.736772
2,1294756200,2294.65991,2296.14990,2279.47998,2288.72998,2242.628247,2188.109552,2110.692317,2010.060131,2240.552490,2193.420806,2080.953206,1986.374322
3,1294842600,2298.62988,2304.87988,2289.05005,2304.87988,2248.556974,2192.688781,2114.537616,2012.993661,2245.166979,2196.950005,2085.744504,1988.058922
4,1294929000,2303.60010,2310.82007,2298.62012,2305.53003,2253.982979,2197.113928,2118.319644,2015.904471,2250.320983,2200.026206,2090.716704,1989.794873
